In [3]:
import os
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable
import torch.optim as optim
from torchvision import datasets, transforms
from models.wideresnet import *
from autoattack import AutoAttack
from models.trades import *

In [4]:
def train(model, device, train_loader, optimizer, epoch, step_size, epsilon, num_steps, beta):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        # calculate robust loss
        loss = trades_loss(model=model,
                           x_natural=data,
                           y=target,
                           optimizer=optimizer,
                           step_size=step_size,
                           epsilon=epsilon,
                           perturb_steps=num_steps,
                           beta=beta,
                           distance='l_inf')
        loss.backward()
        optimizer.step()
        if batch_idx % 5 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [5]:
epochs = 100
step_size = 0.02
epsilon = 0.1
num_steps=10
beta=1

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True}
print("device: ", device)
# set up data loader
transform_test = transforms.Compose([transforms.ToTensor(),])

testset = torchvision.datasets.CIFAR10(root='./data/cifar10', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True, **kwargs)
train_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True, **kwargs)

device:  cuda
Files already downloaded and verified


In [7]:
model = WideResNet(num_classes=10).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(1, epochs):
    train(model, device, train_loader, optimizer, epoch, step_size, epsilon, num_steps, beta)
    test(model, device, test_loader)

if 1:
    torch.save(model.state_dict(), "mnist_cnn.pt")

C:\Users\vladk\anaconda3\envs\torch\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 2.00 GiB total capacity; 1.15 GiB already allocated; 0 bytes free; 1.18 GiB reserved in total by PyTorch)